In [3]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [7]:
from sklearn.pipeline import make_pipeline

In [8]:
import mlflow
from mlflow.tracking import MlflowClient



# mlflow.set_tracking_uri("http://127.0.0.1:5000")
# mlflow.set_experiment("green-taxi-duration")



mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("green-taxi-duration")



<Experiment: artifact_location='/workspaces/mlops-zoomcamp/04-deployment/web-service-mlflow/mlruns/1', creation_time=1746821244632, experiment_id='1', last_update_time=1746821244632, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('/workspaces/mlops-zoomcamp/02-experiment_tracking/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/workspaces/mlops-zoomcamp/02-experiment_tracking/data/green_tripdata_2021-02.parquet')


target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [ ]:
import numpy as np


with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)


    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

    with open('dict_vectorizer.bin', 'wb') as f_out:
        pickle.dump(pipeline[0], f_out)
    mlflow.log_artifact('dict_vectorizer.bin')

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2025/05/09 20:13:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [12]:
from mlflow.tracking import MlflowClient


In [15]:
#MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
RUN_ID = 'bbe9c1128a1742e69de8a87aadf5088b'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [16]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

OSError: No such file or directory: '/workspaces/mlops-zoomcamp/04-deployment/web-service-mlflow/mlruns/1/bbe9c1128a1742e69de8a87aadf5088b/artifacts/dict_vectorizer.bin'

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()